In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [36]:
data_ori = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')

In [31]:
data = data_ori.copy()
len(data)

In [37]:
str_cols = [col for col in data.columns if data[col].dtype == np.dtype('object')]
data[str_cols] = data[str_cols].astype('category')

In [54]:
(data.isna().sum())[data.isna().sum() > 0]

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

In [48]:
data = pd.get_dummies(data)

In [64]:
data = data.fillna(-1)

In [85]:
def feature_processing_baseline(data):
    str_cols = [col for col in data.columns if data[col].dtype == np.dtype('object')]
    data[str_cols] = data[str_cols].astype('category')
    data = pd.get_dummies(data)
    data = data.fillna(-1)
    return data

In [65]:
from sklearn.model_selection import train_test_split

target = 'SalePrice'
features = [col for col in data.columns if col != target]
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target],test_size=0.2)

In [66]:
from sklearn.ensemble import RandomForestRegressor

regr_rf = RandomForestRegressor(n_estimators=100, max_depth=30,
                                random_state=2, oob_score=True)

In [67]:
regr_rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=30, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=2, verbose=0, warm_start=False)

In [68]:
regr_rf.oob_score_

0.8491619528225036

In [69]:
y_rf = regr_rf.predict(X_test)

In [78]:
from sklearn.metrics import mean_squared_error
mean_squared_error(np.log(y_test.to_numpy()), np.log(y_rf))

0.02550938170089764

In [102]:
real_test = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')

In [103]:
real_features = feature_processing_baseline(real_test)[features] # TODO dirty fix doesn't work

KeyError: "['Exterior1st_Stone', 'Condition2_RRNn', 'Exterior2nd_Other', 'GarageQual_Ex', 'Utilities_NoSeWa', 'RoofMatl_ClyTile', 'HouseStyle_2.5Fin', 'Heating_OthW', 'Heating_Floor', 'RoofMatl_Metal', 'MiscFeature_TenC', 'Condition2_RRAn', 'RoofMatl_Roll', 'PoolQC_Fa', 'Exterior1st_ImStucc', 'RoofMatl_Membran', 'Electrical_Mix', 'Condition2_RRAe'] not in index"

In [93]:
real_y = regr_rf.predict(real_features)

In [97]:
result = real_test[['Id']]

In [98]:
result['SalePrice'] = real_y

/Users/dgwang/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
result

,Id,SalePrice
0,1,209454.00
1,2,177304.80
2,3,207592.51
3,4,160159.48
4,5,290611.89
...,...,...
1455,1456,174890.22
1456,1457,206413.74
1457,1458,258602.80
1458,1459,128765.42


In [101]:
result.to_csv('result.csv', index=False)